In [13]:
import numpy as np
import pandas as pd
import tensorflow

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
dataset = pd.read_csv('sense.csv')

In [16]:
dataset

,id,score,review
0,0,Positive,Kokusu çok güzel etkisi de efsane
1,1,Negative,hiç bir fark olmadı siyah nokta silmedi
2,2,Positive,şeffaf güzel bir krem elimi yumuşacık yaptı
3,3,Positive,Özellikle karma ve hassas cildiniz varsa La Ro...
4,4,Negative,çok iyi nemlendiriyor ama daha ucuz muadilleri...
...,...,...,...
149995,149995,Neutral,Suruldugunde hafif bir parlaklik veriyor. Yika...
149996,149996,Positive,Mükemmel nemlendirici hayraniyim bu nemlendiri...
149997,149997,Positive,Çok güzel fiyatı da gayet uygun
149998,149998,Neutral,Fiyat olarak iyi ama düzenli kullansamda geçir...


In [17]:
df = dataset.drop('id', axis=1)

In [18]:
df

,score,review
0,Positive,Kokusu çok güzel etkisi de efsane
1,Negative,hiç bir fark olmadı siyah nokta silmedi
2,Positive,şeffaf güzel bir krem elimi yumuşacık yaptı
3,Positive,Özellikle karma ve hassas cildiniz varsa La Ro...
4,Negative,çok iyi nemlendiriyor ama daha ucuz muadilleri...
...,...,...
149995,Neutral,Suruldugunde hafif bir parlaklik veriyor. Yika...
149996,Positive,Mükemmel nemlendirici hayraniyim bu nemlendiri...
149997,Positive,Çok güzel fiyatı da gayet uygun
149998,Neutral,Fiyat olarak iyi ama düzenli kullansamda geçir...


In [19]:
df['score'].value_counts()

score
Positive    50000
Negative    50000
Neutral     50000
Name: count, dtype: int64

In [20]:
df = pd.get_dummies(df, columns=['score'])

In [21]:
df

,review,score_Negative,score_Neutral,score_Positive
0,Kokusu çok güzel etkisi de efsane,False,False,True
1,hiç bir fark olmadı siyah nokta silmedi,True,False,False
2,şeffaf güzel bir krem elimi yumuşacık yaptı,False,False,True
3,Özellikle karma ve hassas cildiniz varsa La Ro...,False,False,True
4,çok iyi nemlendiriyor ama daha ucuz muadilleri...,True,False,False
...,...,...,...,...
149995,Suruldugunde hafif bir parlaklik veriyor. Yika...,False,True,False
149996,Mükemmel nemlendirici hayraniyim bu nemlendiri...,False,False,True
149997,Çok güzel fiyatı da gayet uygun,False,False,True
149998,Fiyat olarak iyi ama düzenli kullansamda geçir...,False,True,False


In [22]:
df['score_Negative'] = df['score_Negative'].astype(int)
df['score_Neutral'] = df['score_Neutral'].astype(int)
df['score_Positive'] = df['score_Positive'].astype(int)

In [23]:
target = df[['score_Negative', 'score_Neutral', 'score_Positive']].values.tolist()
data = df['review'].values.tolist()

In [24]:
cutoff = int(len(data) * 0.80) #datanın %80 i eğitim için ayrılır#
x_train, x_test = data[:cutoff], data[cutoff:] #cutoff indexine kadar olan kısmı tranine geri kalanı teste atar#
y_train, y_test = target[:cutoff], target[cutoff:]

In [25]:
x_train[500] #traindeki 501. veriyi döndürür, kontrol amaçlı#

'Saçlarda dökülmeyi belirgin derecede azaltıyor. Hem de yumuşacık yapıyor. Kendi yakın arkadaşlarıma dahi önerdiğim bir ürün'

In [26]:
x_train[800]

'paketleme iğrençti'

In [27]:
y_train[800]

[1, 0, 0]

In [28]:
num_words = 30000
#Bu satır, num_words adlı bir değişken oluşturur ve ona 10000 değerini atar.
tokenizer = Tokenizer(num_words=num_words)
#Bu, Tokenizer'ın kullanacağı en fazla kelime sayısını belirtir.
#Yani, veri setindeki en sık kullanılan 10000 kelimeye odaklanılacak ve geri kalanı dikkate alınmayacak.#

In [29]:
tokenizer.fit_on_texts(data)

**tokenizer.fit_on_texts(data)** satırı, Tokenizer nesnesini eğitir. Bu, Tokenizer'ın veri setindeki her bir kelimenin sıklığını hesaplamasını ve bu bilgiyi daha sonra metinleri sayılara dönüştürmek için kullanmasını sağlar.

**fit_on_texts metodu**, verilen metin verilerini alır (data), her bir kelimenin ne kadar sık kullanıldığını hesaplar ve bu bilgiyi iç dahili sözlüklerde saklar. Bu sözlükler, daha sonra metinleri sayılara dönüştürmek için kullanılır.

Bu adım, metin verilerini bir sinir ağı modeline girebilecek bir formata dönüştürme sürecinin bir parçasıdır. Bu süreç genellikle "tokenleştirme" olarak adlandırılır ve metinleri bireysel kelimelere ("tokenler") ayırır ve daha sonra bu kelimeleri sayılara dönüştürür. Bu, bir sinir ağı modelinin metin verilerini işleyebilmesini sağlar.

In [30]:
tokenizer.word_index #kelimeleri indexlerine göre sıralar (Çok kullanım indexine göre)#

{'çok': 1,
 'bir': 2,
 'güzel': 3,
 'ürün': 4,
 've': 5,
 'ama': 6,
 'değil': 7,
 'gibi': 8,
 'iyi': 9,
 'daha': 10,
 'için': 11,
 'bu': 12,
 'hiç': 13,
 'kokusu': 14,
 'geldi': 15,
 'tavsiye': 16,
 'da': 17,
 'aldım': 18,
 'de': 19,
 'uygun': 20,
 'yok': 21,
 'var': 22,
 'kullanıyorum': 23,
 'kötü': 24,
 'küçük': 25,
 'beğenmedim': 26,
 'kadar': 27,
 'gayet': 28,
 'sonra': 29,
 'kesinlikle': 30,
 'ben': 31,
 'fakat': 32,
 'biraz': 33,
 'memnun': 34,
 'işe': 35,
 'oldu': 36,
 'ürünü': 37,
 'ederim': 38,
 'göre': 39,
 'ilk': 40,
 'bile': 41,
 'büyük': 42,
 'harika': 43,
 'rengi': 44,
 'olarak': 45,
 'o': 46,
 'hızlı': 47,
 'iade': 48,
 'gerçekten': 49,
 'fiyatı': 50,
 'uzun': 51,
 'pek': 52,
 '2': 53,
 'beğendim': 54,
 '1': 55,
 'fazla': 56,
 'aşırı': 57,
 'en': 58,
 'teşekkürler': 59,
 'yumuşacık': 60,
 'kullandığım': 61,
 'etkisini': 62,
 'bence': 63,
 'kullandım': 64,
 'ürünün': 65,
 'yaptı': 66,
 'fiyat': 67,
 'yapıyor': 68,
 'etkisi': 69,
 'hemen': 70,
 'sadece': 71,
 'hem': 72,
 '

In [31]:
x_train_tokens = tokenizer.texts_to_sequences(x_train) #metinleri sayı dizilerine dönüştürür#

In [32]:
x_train[800]

'paketleme iğrençti'

In [34]:
print(x_train_tokens[800])

[95, 19274]


In [35]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [36]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
#Her bir yorumdaki kelime sayısını hesaplar ve bir listeye ekler.#
num_tokens = np.array(num_tokens)
#bu kod parçası, her bir token dizisinin uzunluğunu hesaplar ve bu uzunlukları bir NumPy dizisine dönüştürür.#

In [37]:
np.mean(num_tokens)
#num_tokens NumPy dizisindeki tüm değerlerin ortalamasını hesaplar.#

11.60962

In [38]:
np.max(num_tokens)

273

In [39]:
np.argmax(num_tokens)
#bu bilgiyi kullanarak, en uzun token dizisini bulabilir ve
# bu dizinin uzunluğunu, diğer tüm dizileri doldurmak veya kırpmak için bir maksimum uzunluk olarak kullanabilirsiniz#

9782

In [40]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens
#Bu kod parçası, num_tokens dizisindeki değerlerin ortalamasını ve standart sapmasını kullanarak
#bir max_tokens değeri hesaplar ve bu değeri bir tam sayıya dönüştürür#

31

In [41]:
np.sum(num_tokens < max_tokens) / len(num_tokens)
#Bu kod parçası, num_tokens dizisindeki token dizilerinin yüzde kaçının max_tokens değerinden daha kısa olduğunu hesaplar.#

0.9521866666666666

In [42]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [43]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

Bu satırlar, pad_sequences fonksiyonunu kullanarak x_train_tokens listesindeki her bir token dizisini max_tokens uzunluğuna getirir ve sonucu x_train_pad adlı bir değişkene atar. Padding ekler veya kırpar.

In [44]:
x_train_pad.shape

(120000, 31)

In [45]:
x_test_pad.shape

(30000, 31)

In [46]:
np.array(x_train_tokens[800])

array([   95, 19274])

In [47]:
x_train_pad[800]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,    95, 19274], dtype=int32)

In [48]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

Bu kod parçası, bir kelime indeksi (word_index) oluşturur ve ardından bu indeksin tersini alır.

Bu satır, tokenizer nesnesinin word_index özelliğini alır ve idx adlı bir değişkene atar. word_index özelliği, her bir benzersiz kelimenin indeksini içeren bir sözlük döndürür. Bu indeks, genellikle bir sinir ağı modeline girecek metin verilerini hazırlarken kullanılır.

Bu satır, idx sözlüğünün tersini alır ve inverse_map adlı bir değişkene atar. Bu, idx sözlüğünün değerlerini ve anahtarlarını alır, bunları birlikte gruplar (zip) ve sonucu bir sözlüğe dönüştürür (dict). Bu, genellikle bir sinir ağı modelinden çıkan verileri yorumlamak için kullanılır, çünkü model genellikle indekslerle çalışır ve bu indeksleri geri kelimelere dönüştürmek için bir ters harita gereklidir.

In [49]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [50]:
x_train[800]

'paketleme iğrençti'

In [51]:
tokens_to_string(x_train_tokens[800])

'paketleme iğrençti'

In [52]:
model = Sequential()

In [53]:
embedding_size = 200

#embedding_size = 200 ifadesi, kelime gömme (word embedding) vektörlerinin boyutunu belirler.
#Kelime gömme, doğal dil işleme (NLP) görevlerinde sıklıkla kullanılan bir tekniktir.
#Bu teknik, her bir kelimenin bir vektör temsilini oluşturur. Bu vektörler, genellikle bir sinir ağı modeline girecek metin verilerini hazırlarken kullanılır.
#embedding_size değişkeni, her bir kelime gömme vektörünün boyutunu belirler. Bu durumda, her bir kelime gömme vektörü 50 boyutlu olacaktır.
#Bu değer, genellikle bir hiperparametre olarak kabul edilir ve modelin performansını etkileyebilir.
#Bu değer çok küçük olduğunda, model kelimeler arasındaki tüm nüansları öğrenmekte zorlanabilir.
#Ancak bu değer çok büyük olduğunda, modelin eğitilmesi daha zor ve zaman alıcı olabilir.
#Bu yüzden, bu değeri belirlerken bir denge bulmak önemlidir.#


In [54]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

Embedding katmanı, doğal dil işleme (NLP) görevlerinde sıklıkla kullanılan bir tekniktir. Bu katman, her bir kelimenin bir vektör temsilini oluşturur. Bu vektörler, genellikle bir sinir ağı modeline girecek metin verilerini hazırlarken kullanılır.

input_dim=num_words: Bu parametre, kelime gömme katmanının girdi boyutunu belirler. Bu genellikle kelime dağarcığınızdaki toplam kelime sayısıdır (num_words).

output_dim=embedding_size: Bu parametre, kelime gömme vektörlerinin boyutunu belirler (embedding_size). Bu, her bir kelime gömme vektörünün boyutudur.

input_length=max_tokens: Bu parametre, her bir girdi dizisinin (örneğin bir cümle) uzunluğunu belirler. Bu genellikle en uzun cümlenin token sayısıdır (max_tokens).

name='embedding_layer': Bu parametre, bu katmana bir isim atar. Bu isim, modelin diğer katmanlarından bu katmanı ayırt etmek için kullanılabilir.

Bu katman, modelin ilk katmanı olarak eklenir ve metin verilerini vektör formata dönüştürür. Bu vektörler, modelin sonraki katmanlarına girdi olarak verilir.

In [57]:
model.add(LSTM(units=16, return_sequences=True))
model.add(LSTM(units=8, return_sequences=True))
model.add(LSTM(units=4))
model.add(Dense(3, activation='softmax'))

In [58]:
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.001)

In [59]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 31, 200)           6000000   
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 31, 16)            13888     
                                                                 
 lstm_1 (LSTM)               (None, 31, 8)             800       
                                                                 
 lstm_2 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 3)                 15        
                                                                 
Total params: 6014911 (22.95 MB)
Trainable params: 6014911 (22.95 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [61]:
y_train = np.array(y_train)

In [62]:
model.fit(x_train_pad, y_train, epochs=10, batch_size=512)

Epoch 1/10
235/235 [==============================] - 51s 193ms/step - loss: 0.5593 - accuracy: 0.5794
Epoch 2/10
235/235 [==============================] - 46s 195ms/step - loss: 0.4120 - accuracy: 0.7743
Epoch 3/10
235/235 [==============================] - 46s 194ms/step - loss: 0.3396 - accuracy: 0.8232
Epoch 4/10
235/235 [==============================] - 46s 194ms/step - loss: 0.2979 - accuracy: 0.8470
Epoch 5/10
235/235 [==============================] - 47s 201ms/step - loss: 0.2688 - accuracy: 0.8639
Epoch 6/10
235/235 [==============================] - 45s 191ms/step - loss: 0.2473 - accuracy: 0.8755
Epoch 7/10
235/235 [==============================] - 46s 195ms/step - loss: 0.2305 - accuracy: 0.8843
Epoch 8/10
235/235 [==============================] - 45s 192ms/step - loss: 0.2154 - accuracy: 0.8935
Epoch 9/10
235/235 [==============================] - 45s 192ms/step - loss: 0.2033 - accuracy: 0.9002
Epoch 10/10
235/235 [==============================] - 48s 203ms/step - l

In [63]:
y_test = np.array(y_test)

In [64]:
result = model.evaluate(x_test_pad, y_test)

938/938 [==============================] - 11s 10ms/step - loss: 0.3650 - accuracy: 0.7839


In [65]:
result[1]

0.7839000225067139

In [66]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

32/32 [==============================] - 1s 9ms/step


In [80]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [81]:
cls_true = np.array(y_test[0:1000])

In [82]:
cls_true = np.argmax(cls_true, axis=1)

In [83]:
incorrect_indices = np.where(cls_pred != cls_true)[0]

In [79]:
print(cls_pred.shape)
print(cls_true.shape)

(1000,)
(1000,)


In [84]:
len(incorrect_indices)

887

In [85]:
idx = incorrect_indices[0]
idx

0

In [86]:
text = x_test[idx]
text

'pek memnun kalmadık kutular /20 boş  kalitesinde'

In [87]:
y_pred[idx]

0.9983247

In [88]:
cls_true[idx]

0

In [89]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "stabil"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [90]:
tokens = tokenizer.texts_to_sequences(texts)

In [91]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(10, 31)

In [92]:
model.predict(tokens_pad)

1/1 [==============================] - 0s 37ms/step


array([[1.16533646e-03, 6.42024353e-03, 9.92414355e-01],
       [1.67493932e-02, 4.29149002e-01, 5.54101586e-01],
       [9.97762740e-01, 1.85253518e-03, 3.84662242e-04],
       [8.49661410e-01, 1.45175815e-01, 5.16268006e-03],
       [7.94541478e-01, 2.04759777e-01, 6.98845135e-04],
       [9.98546541e-01, 1.09912571e-03, 3.54268414e-04],
       [1.13384664e-01, 5.88110745e-01, 2.98504591e-01],
       [9.98584151e-01, 1.06878788e-03, 3.47088557e-04],
       [1.85484800e-03, 1.42295156e-02, 9.83915627e-01],
       [8.44813466e-01, 1.52399898e-01, 2.78661773e-03]], dtype=float32)